In [1]:
def timer(func):
    import time
    def wrapper(*args, **kwargs):
        start = time.time()
        func(*args, **kwargs)
        end = time.time()
        print('Time : {:.10e}'.format(end - start))
    return wrapper

# 상하좌우

In [2]:
input1 = '''5
R R R U D D'''

@timer
def solution(input1):
    N = int(input1.split('\n')[0])
    array = input1.split('\n')[1:][0].split()

    loc = [1, 1]
    for cmd in array:
        if cmd == 'L' and loc[1] > 1:
            loc[1] -= 1
        elif cmd == 'R' and loc[1] < N:
            loc[1] += 1
        elif cmd == 'U' and loc[0] > 1:
            loc[0] -= 1
        elif cmd == 'D' and loc[0] < N:
            loc[0] += 1

    print(' '.join(list(map(str, loc))))
    
solution(input1)

3 4
Time : 0.0000000000e+00


In [3]:
@timer
def solution(input1):
    N = int(input1.split('\n')[0])
    plans = input1.split('\n')[1:][0].split()

    x, y = 1, 1
    dx = [0, 0, -1, 1]
    dy = [-1, 1, 0, 0]
    move_types = ['L', 'R', 'U', 'D']

    for plan in plans:
        for i in range(len(move_types)):
            if plan == move_types[i]:
                nx = x + dx[i]
                ny = y + dy[i]
        if nx < 1 or ny < 1 or nx > N or ny > N:
            continue

        x, y = nx, ny

    print(x, y)
    
solution(input1)

3 4
Time : 0.0000000000e+00


# 시각

In [18]:
input1 = '''5'''

@timer
def solution(input1):
    N = int(input1)
    cnt = 0
    s = 0
    m = 0
    h = 0
    while h <= N:
        if '3' in str(h) or '3' in str(m) or '3' in str(s):
           cnt += 1 
        s += 1
        if s == 60:
            s = 0
            m += 1
        if m == 60:
            m = 0
            h += 1

    print(cnt)

solution(input1)

11475
Time : 1.1959791183e-02


In [19]:
@timer
def solution(input1):
    N = int(input1)
    cnt = 0

    for i in range(60):
        for j in range(60):
            for k in range(N + 1):
                time = str(k) + str(j) + str(i)
                if '3' in time:
                    cnt += 1
    print(cnt)
    
solution(input1)

11475
Time : 1.5005350113e-02


# 왕실의 나이트

In [103]:
input1 = 'a1' # 2
input2 = 'c2' # 6

@timer
def solution(input1):
    import copy
    
    src = list(input1)
    
    # index로 숫자 매핑 
    al = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    num = list(map(str, [1, 2, 3, 4, 5, 6, 7, 8]))
    assert src[0] in al and src[1] in num, 'invalid location info'
    
    src[0] = str(num[al.index(src[0])])
    src = list(map(int, src))
    
#     # ord 함수는 알파벳의 유니코드를 반환
#     src[0] = ord(src[0]) - ord('a') + 1
#     src[1] = int(src[1])

    cnt = 0
    # (+2, +1), (+2, -1), (-2, +1), (-2, -1), (+1, +2), (-1, +2), (+1, -2), (-1, -2)
    # x, y좌표가 각각 +2와 -2, +1와 -1로 구성 , x, y 좌표 자리를 바꿀 시 모든 경우의 수
    for x in [2, -2]:
        for y in [1, -1]:
            for i in [True, False]:
            # copy하지 않을 경우 src가 수정되어 버림
                tmp = copy.copy(src)
                tmp[int(i)] += x
                tmp[int(not i)] += y
            # 체스판 위를 나가지 않는 경우만 append
                if 1 <= tmp[0] <= 8 and 1 <= tmp[1] <= 8:
                    cnt += 1
                    
    print(cnt)
    
solution(input1)

solution(input2)

2
Time : 0.0000000000e+00
6
Time : 0.0000000000e+00


# 게임 개발
- 순환하는 수열의 경우 나머지를 이요하면 편리
- 방향 이동 문제의 경우, 이동 경우의 수를 리스트로 관리하면 편리

In [171]:
input1 = '4 4'
input2 = '1 1 0'
input3 = '''1 1 1 1
1 0 0 1
1 1 0 1
1 1 1 1'''

input4 = '3 3'
input5 = '1 1 0'
input6 = '''1 1 0
1 1 1
1 0 0
1 1 0'''

In [170]:
@timer
def solution(input1, input2, input3):
    N, M = list(map(int, input1.split()))
    loc = list(map(int, input2.split()[:2]))
    # 0 = 위, 1 = 오른쪽, 2 = 아래, 3 = 왼쪽
    direction = int(input2.split()[2])
    moves = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    field = [list(map(int, m.split())) for m in input3.split('\n')]
    
    # 현재 위치를 지나간 곳으로 체크
    field[loc[0]][loc[1]] = -1
    cnt = 1

    # 현재 위치 기준 사방을 확인하는 함수 정의, 모두 가보지 않은 곳이면 False 반환
    def check_around(loc):
        down = field[loc[0] - 1][loc[1]] != 0
        up = field[loc[0] + 1][loc[1]] != 0
        left = field[loc[0]][loc[1] - 1] != 0 
        right = field[loc[0]][loc[1] + 1] != 0
        return False if down + up + left + right == 4 else True

    while True:
        # 1단계 : 왼쪽 회전 후 방향에 따른 이동거리 정의
        direction = (direction - 1) % 4
        dy = moves[direction][0]
        dx = moves[direction][1]
        new_loc = (loc[0] + dy, loc[1] + dx)
        # 2단계 : 왼쪽이 가보지 않은 곳일 경우 위치를 이동, 카운팅
        if field[new_loc[0]][new_loc[1]] == 0:
            field[new_loc[0]][new_loc[1]] = -1
            cnt += 1
            loc = new_loc
            continue
        # 3단계 : 사방이 다 바다 혹은 가본 곳일 경우 뒤로 이동
        if not check_around(loc):
            dy = -(moves[direction][0])
            dx = -(moves[direction][1])
            new_loc = (loc[0] + dy, loc[1] + dx)
            # 뒤가 바다일 경우 종료
            if field[new_loc[0]][new_loc[1]] == 1:
                break
            loc = new_loc
            continue
            
    print(cnt)

solution(input1, input2, input3)

3
Time : 0.0000000000e+00


In [200]:
@timer
def solution(input1, input2, input3):
    N, M = list(map(int, input1.split()))
    x, y, direction = list(map(int, input2.split()))

    dx = [-1, 0, 1, 0]
    dy = [0, 1, 0, -1]

    path = [[0] * M for _ in range(N)]
    path[x][y] = 1
    field = [list(map(int, row.split())) for row in input3.split('\n')]

    cnt = 1
    n_turn = 0
    while True:
        direction = (direction - 1) % 4
        nx = x + dx[direction]
        ny = y + dy[direction]
        if field[nx][ny] == 0 and path[nx][ny] ==0:
            path[nx][ny] = 1
            x, y = nx, ny
            cnt += 1
            n_turn = 0
            continue
        else:
            n_turn += 1

        if n_turn == 4:
            nx = x - dx[direction]
            ny = y - dy[direction]
            if field[nx][ny] == 1:
                break
            x = nx
            y = ny
            n_turn = 0

    print(cnt)        
    
solution(input1, input2, input3)

3
Time : 0.0000000000e+00


# 아기 상어
- N×N 크기의 공간에 물고기 M마리와 아기 상어 1마리가 있다. 공간은 1×1 크기의 정사각형 칸으로 나누어져 있다. 한 칸에는 물고기가 최대 1마리 존재한다.

아기 상어와 물고기는 모두 크기를 가지고 있고, 이 크기는 자연수이다. 가장 처음에 아기 상어의 크기는 2이고, 아기 상어는 1초에 상하좌우로 인접한 한 칸씩 이동한다.

아기 상어는 자신의 크기보다 큰 물고기가 있는 칸은 지나갈 수 없고, 나머지 칸은 모두 지나갈 수 있다. 아기 상어는 자신의 크기보다 작은 물고기만 먹을 수 있다. 따라서, 크기가 같은 물고기는 먹을 수 없지만, 그 물고기가 있는 칸은 지나갈 수 있다.

아기 상어가 어디로 이동할지 결정하는 방법은 아래와 같다.

더 이상 먹을 수 있는 물고기가 공간에 없다면 아기 상어는 엄마 상어에게 도움을 요청한다.
먹을 수 있는 물고기가 1마리라면, 그 물고기를 먹으러 간다.
먹을 수 있는 물고기가 1마리보다 많다면, 거리가 가장 가까운 물고기를 먹으러 간다.
거리는 아기 상어가 있는 칸에서 물고기가 있는 칸으로 이동할 때, 지나야하는 칸의 개수의 최솟값이다.
거리가 가까운 물고기가 많다면, 가장 위에 있는 물고기, 그러한 물고기가 여러마리라면, 가장 왼쪽에 있는 물고기를 먹는다.
아기 상어의 이동은 1초 걸리고, 물고기를 먹는데 걸리는 시간은 없다고 가정한다. 즉, 아기 상어가 먹을 수 있는 물고기가 있는 칸으로 이동했다면, 이동과 동시에 물고기를 먹는다. 물고기를 먹으면, 그 칸은 빈 칸이 된다.

아기 상어는 자신의 크기와 같은 수의 물고기를 먹을 때 마다 크기가 1 증가한다. 예를 들어, 크기가 2인 아기 상어는 물고기를 2마리 먹으면 크기가 3이 된다.

공간의 상태가 주어졌을 때, 아기 상어가 몇 초 동안 엄마 상어에게 도움을 요청하지 않고 물고기를 잡아먹을 수 있는지 구하는 프로그램을 작성하시오.
- 첫째 줄에 공간의 크기 N(2 ≤ N ≤ 20)이 주어진다.

둘째 줄부터 N개의 줄에 공간의 상태가 주어진다. 공간의 상태는 0, 1, 2, 3, 4, 5, 6, 9로 이루어져 있고, 아래와 같은 의미를 가진다.

0: 빈 칸
1, 2, 3, 4, 5, 6: 칸에 있는 물고기의 크기
9: 아기 상어의 위치
아기 상어는 공간에 한 마리 있다.

In [2]:
input1 = '''3
0 0 0
0 0 0
0 9 0''' # 0
input2 = '''3
0 0 1
0 0 0
0 9 0'''# 3
input3 = '''4
4 3 2 1
0 0 0 0
0 0 9 0
1 2 3 4''' #14
input4 = '''6
5 4 3 2 3 4
4 3 2 3 4 5
3 2 9 5 6 6
2 1 2 3 4 5
3 2 1 6 5 4
6 6 6 6 6 6''' #60
input5 = '''6
6 0 6 0 6 1
0 0 0 0 0 2
2 3 4 5 6 6
0 0 0 0 0 2
0 2 0 0 0 0
3 9 3 0 0 1''' #48
input6 = '''6
1 1 1 1 1 1
2 2 6 2 2 3
2 2 5 2 2 3
2 2 2 4 6 3
0 0 0 0 0 6
0 0 0 0 0 9''' #39

In [3]:
from collections import deque

N = int(input2.split('\n')[0])
graph = [list(map(int, row.split())) for row in input2.split('\n')[1:]]
visited = [[0] * N for _ in range(N)]

size = 2
feed = 0

# bfs를 사용하여 한칸씩 확장해가면서 가장 먼저 만나는 물고기를 먹는다.

# 상 하 좌 우
dx = [-1, 1, 0, 0]
dy = [0, 0, -1, 1]

def bfs(x, y):
    global feed
    visited[x][y] = 1
    dist = 0
    
    queue = deque()
    queue.append((x, y))
    
    while deque:
        print(np.array(visited))
        x, y = queue.popleft()
        for i in range(4):
            nx = x + dx[i]
            ny = y + dy[i]
            if nx < 0 or ny < 0 or nx >= N or ny >= N:
                continue
            if graph[nx][ny] == 0 and visited[nx][ny] == 0:
                visited[nx][ny] = visited[x][y] + 1
                queue.append((nx, ny))
                dist += 1
            if graph[nx][ny] < size and graph[nx][ny] != 0:
                feed += 1
                return dist
bfs(2, 1)                

[[0 0 0]
 [0 0 0]
 [0 1 0]]
[[0 0 0]
 [0 2 0]
 [2 1 2]]
[[0 3 0]
 [3 2 3]
 [2 1 2]]
[[0 3 0]
 [3 2 3]
 [2 1 2]]
[[0 3 0]
 [3 2 3]
 [2 1 2]]


7

In [5]:
from collections import deque

N = int(input2.split('\n')[0])
graph = [list(map(int, row.split())) for row in input2.split('\n')[1:]]
visited = [[0] * N for _ in range(N)]

size = 2
feed = 0

# dfs를 사용하여 재귀적으로 함수를 관리하여 최단거리를 찾음
dist = 0
def dfs(x, y):
    global dist
    dist += 1
    visited[x][y] = 1
    if graph[x][y] < size and graph[x][y] != 0:
        return dist
    if x < 0 or y < 0 or x >= N or y >= N:
        pass
    if graph[x][y] <= size and visited[x][y] == 0:
        graph[x][y] = 987654321
        dfs(x + 1, y)
        dfs(x - 1, y)
        dfs(x, y + 1)
        dfs(x, y - 1)
        return x, y
        
        
dfs(2, 1)

In [331]:
def solution(input2):
    N = int(input2.split('\n')[0])
    field = [list(map(int, row.split())) for row in input2.split('\n')[1:]]
    size = 2
    feed = 0
    total_dist = 0
    
    for n in range(N):
        if 9 in field[n]:
            x, y = n, field[n].index(9)
            
    while True:
        min_dist = 99999

        # 거리 구하기
        for i in range(N):
            for j in range(N):
                target = field[i][j]
                # 물고기가 존재하는 좌표에 대해서 좌표와 거리를 저장
                if target != 0 and target != 9 and target < size:
                    dist = abs(x - i) + abs(y - j)
                    # 최단거리인 경우, 먹을 수 있는 경우를 저장
                    if dist < min_dist:
                        min_dist = dist
                        nx, ny = i, j

        if min_dist == 99999:
            break
        total_dist += min_dist
        x, y = nx, ny
        field[x][y] = 0

        feed += 1
        if feed == size:
            feed = 0
            size += 1
            continue
        
    print(total_dist)

solution(input1)    
solution(input2)
solution(input3)
solution(input4)
solution(input5) # X
solution(input6) # X
# 경로상에 큰 물고기가 가로막고있는 경우를 처리해야 함

0
3
14
60
28
33


# 럭키 스트레이트

In [15]:
input1 = '''123402'''
input2 = '''7755'''

def solution(input1):
    array = list(map(int, input1))
    mid = len(array) // 2
    print('LUCKY' if sum(array[:mid]) == sum(array[mid:]) else 'READY')

solution(input1)
solution(input2)

LUCKY
READY


# 문자열 재정렬

In [31]:
input1 = '''K1KA5CB7'''
input2 = '''AJKDLSI412K4JSJ9D'''

def solution(input1):
    array = list(map(str, input1))
    str_array = []
    int_array = []

    for i in array:
        try:
            int_array.append(int(i))
        except:
            str_array.append((ord(i) - ord('A') + 1, i))

    result = []
    str_array.sort()
    for _, i in str_array:
        result.append(i)
    result.append(str(sum(int_array)))
    print(''.join(result))
    
solution(input1)
solution(input2)

ABCKK13
ADDIJJJKKLSS20


# 문자열 압축

In [163]:
input1 = '''aabbaccc'''
input2 = '''ababcdcdababcdcd'''
input3 = '''abcabcdede'''
input4 = '''abcabcabcabcdededededede'''
input5 = '''xababcdcdababcdcd'''
input6 = '''abcdefgabcdefg'''


def solution(input1):
    s = input1
    length = len(s)
    compressed = ''
    cnt = 1
    min_cnt = 987654321
    
    for n in range(1, length // 2 + 1):
        ngrams = []
        for i in range(0, length, n):
            if s[i:i + n] != '':
                ngrams.append(s[i:i + n])
        print(ngrams)
        for i in range(len(ngrams) - 1):
            if ngrams[i] == ngrams[i + 1]:
                cnt += 1
            else:
                if cnt > 1:
                    compressed += str(cnt) + ngrams[i]
                    cnt = 1
                else:
                    compressed += ngrams[i]

        if cnt > 1:
            compressed += str(cnt) + ngrams[-1]
            cnt = 1
        else:
            compressed += ngrams[-1]

        min_cnt = min(min_cnt, len(compressed))
        compressed = ''

    print(min_cnt)
    
solution(input1)
solution(input2)
solution(input3)
solution(input4)
solution(input5)
solution(input6)

['a', 'a', 'b', 'b', 'a', 'c', 'c', 'c']
['aa', 'bb', 'ac', 'cc']
['aab', 'bac', 'cc']
['aabb', 'accc']
7
['a', 'b', 'a', 'b', 'c', 'd', 'c', 'd', 'a', 'b', 'a', 'b', 'c', 'd', 'c', 'd']
['ab', 'ab', 'cd', 'cd', 'ab', 'ab', 'cd', 'cd']
['aba', 'bcd', 'cda', 'bab', 'cdc', 'd']
['abab', 'cdcd', 'abab', 'cdcd']
['ababc', 'dcdab', 'abcdc', 'd']
['ababcd', 'cdabab', 'cdcd']
['ababcdc', 'dababcd', 'cd']
['ababcdcd', 'ababcdcd']
9
['a', 'b', 'c', 'a', 'b', 'c', 'd', 'e', 'd', 'e']
['ab', 'ca', 'bc', 'de', 'de']
['abc', 'abc', 'ded', 'e']
['abca', 'bcde', 'de']
['abcab', 'cdede']
8
['a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'd', 'e', 'd', 'e', 'd', 'e', 'd', 'e', 'd', 'e', 'd', 'e']
['ab', 'ca', 'bc', 'ab', 'ca', 'bc', 'de', 'de', 'de', 'de', 'de', 'de']
['abc', 'abc', 'abc', 'abc', 'ded', 'ede', 'ded', 'ede']
['abca', 'bcab', 'cabc', 'dede', 'dede', 'dede']
['abcab', 'cabca', 'bcded', 'edede', 'dede']
['abcabc', 'abcabc', 'dedede', 'dedede']
['abcabca', 'bcabcde', 'dededed',

In [131]:
    ngrams

['aabb', 'accc']